# Agricultural Supply Chains


# A framework for constructing and simulating rule-based models.

Before starting development on the supply chain model, I wanted to develop a framework that made creating the rules, locations ect of the model easy, as well as code to validate the correctness of model definition and a way to separate the model creation and the model simulation code.

The framework should work to build a variety of models, we illustrate that here by building a stochastic SEIR model with a spatial-based transmission propensity (pending development work).

## Building Blocks

### Classes

The individual compartments of the model, defined by a name, a unit and an optional flag to indicate what type of restrictions to apply to the class (Work in progress, but e.g. should be an integer). By defining a class with a unit ...

In [ ]:
epiClasses = [["S", "people (thousands)"], ["E", "people (thousands)"], ["I", "people (thousands)"], ["R", "people (thousands)"]]

### Locations 

Locations serve to group together different classes and form a core part of the "rule matching" system. 

Comprised of a set of a classes, a type. Optionally each location type has constants and variables set at an instance level and defined at a type/class level.

By recording lat/long of each location, we can (and do) compute all pairwise distances, at the moment using the Haversine formula but this could for example be fed from Google Maps.

In [ ]:
# We define our location classes.
class Region (ModelLocations.Location):
    def __init__(self, lat, long, name):
        # Sets lat/long and creates and empty set of compartment labels.
        super().__init__(lat, long, name, loc_type="Region")

        self.class_labels.add("S")
        self.class_labels.add("E")
        self.class_labels.add("I")
        self.class_labels.add("R")

In [ ]:
# We define our different locations

# There is great flexibility, as long as the function returns an array of Locations (only require the parent class is Location).

def epiLocations():
    all_locations = []
    # Midpoints from wikipedia, South East and London uses South East midpoint (combined to match DEFRA reporting)
    region_infos = [[0, 0, "Test Region"], [54.075, -2.75, "North East"], [55, -1.87, "North West"], [53.566667, -1.2, "Yorkshire & The Humber"], 
                    [52.98, -0.75, "East Midlands"], [52.478861, -2.256306, "West Midlands"], 
                    [52.24, 0.41, "East of England"], [51.515447, -0.09214, "London"], [51.3, -0.8, "South East"], 
                    [50.96, -3.22, "South West"], [56.816738, -4.183963, "Scotland"], 
                    [52.33022, -3.766409,"Wales"]]
    # ONS mid 2022 estimates
    region_populations = list(0.001*np.array([100000, 2683040, 7516113, 5541262, 4934939, 6021653, 6398497, 8866180, 
                                              9379833, 5764881, 5447700, 3131640]))

    for region_index, region_info in enumerate(region_infos):
        # Creates a region from each array entry in region infos
        region = Region(*region_info)
        # Sets 1000 infected people in every region and everyone else as susceptible
        region.setInitialConditions({"S":region_populations[region_index]-1,"I":1})
        all_locations.append(region)

    
    return all_locations

### Rules 

Comprised of each of a propensity, stochiometry and a type for any number of locations. At the moment the propensities for each location are mutiplied together. Stochiometry is a per location numpy array and propensity is a per location sympy formula for each class. Symbols supported include any class present at the location, model state information (e.g. model_month_mar which is an indicator variable for March, these can be chained together to allow for seasonality) and constants/functions supported by sympy

In [ ]:
model_constants = {
    "Exposure_rate":2,
    "Infection_rate":1,
    "Recovery_rate":0.2,
    "Death_proportion":0.01,
    "Standard_Transport_prop":0.001,
    "Infected_Transport_prop":0.0001
}

In [ ]:
# Here we rely on prefilled SingleLocationProductRule, ExitEntranceRule, TransportRule classes to simplify the creation of rules

# There is great flexibility, as long as the function returns an array of Rules.

def epiRules():
    # Frequency vs prevalance - /N but can also use a frequency propensity (fits some dieases better than others).
    # Here we observe model_month_jan and model_month_feb, these indicators can be used to create a seasonal effect in the infectivity of a diease. 
    exposure = SupplyChainRules.SingleLocationProductionRule("Region", ["S"], [1], ["E"], [1], 
                                                              f"{model_constants['Exposure_rate']}*I*S/(S+E+I+R)*model_month_jan +\
                                                                {model_constants['Exposure_rate']}*I*S/(S+E+I+R)*model_month_feb",
                                                              ["S","E","I","R"], "Exposure")
    infection = SupplyChainRules.SingleLocationProductionRule("Region", ["E"], [1], ["I"], [1], 
                                                              f"{model_constants['Infection_rate']}*E",
                                                              ["E"], "Infection")    
    recovery = SupplyChainRules.SingleLocationProductionRule("Region", ["I"], [1], ["R"], [1],
                                                            f"{model_constants['Recovery_rate']}*I",
                                                            ["I"], "Recovery")
    death = SupplyChainRules.ExitEntranceRule("Region", "I", 1, 
                                              f"{model_constants['Death_proportion']}*{model_constants['Recovery_rate']}*I", ["I"], "Death")
    
    transports = []
    # Here we describe the rules that govern the movement of each infection class between regions.
    # The rule matching systems will create 66*4 rules for each ordered pair of regions of the UK for all infection states.
    for transport in ["S","E","I","R"]:
        if transport == "I":
            relocation_rate = model_constants["Infected_Transport_prop"]
        else:
            relocation_rate = model_constants["Standard_Transport_prop"]

        transports.append(SupplyChainRules.TransportRule("Region", "Region", transport, [f"{str(relocation_rate)}*{transport}","1"], 
                                                     1, [[transport], ["S"]], "Movement"))
        
    return [exposure, infection, recovery, death] + transports

#### Putting everything together

In [ ]:
model = ModelDefinition.ModelDefinition(epiClasses, epiLocations, epiRules, model_folder="../ModelFiles/RegionalEpi/")
model.build()

State information vs classes - while state information such as time, could be encoded as a class and the dynamics could be encoded as rules, it could make sense to conceptually separate quantities such as time, particularly as this is shared across the model, the time increment isn't constant and 


## Construction

The model construction is programmatic at the moment - with further work, a model could be created interactively using a UI. 

Basic checks ensure that model construction is correct (locations have classes required by rules, locations exists such that the rule could be triggered, the propensity is a well defined formula, all classes are defined).

"Rule matching", rather than defining a rule between all pairs, triplets, ect of locations we can define a "meta-rule". Each location has a type associated, and each rule has a type for each location slot. All permutations of locations that match the location slots are found, and each of these form a possible way the rule can be triggered.

We note that at the moment the order of the location matters - support may be added for rule matching without consideration of the order of the matching.


## Simulation

Currently implemented the Gillespie algorithm, aim to implement atleast the Tau-leaping algorithm.

Model state information - allows 

Maybe have location specific state information?

Returns a trajectory object - only basic plotting at the moment but plan to add significantly more.

In [ ]:
import Application
import datetime

start_date = datetime.datetime(2001, 1, 1)

epi_application = Application.ModelBackend(start_date=start_date, model_folder="../ModelFiles/RegionalEpi/")
out = epi_application.simulate(time_limit=365)

out.plotAllClassesOverTime(location_index=0)

# An example model: UK Crop Agricultural Supply Chains

# Crops

We split the crops into wheat, barley ect and further into the stage of plant growth, from unplanted seed into harvested plant, at each stage, we will model the growth as well as the wastage - (e.g failure to germinate, plant death).

There will be a large emphasis on the seasonality of each of the stages aswell as the weather/growing condiditions.

Additionally, we will try to model the impact of soil qualtiy/health on growing conditions with an emphasis on Nitrogen 

# Weather

Weather is extremely important to agriculture and this is particularly interesting to look at with the ongoing trend of climate change.

# Fertiliser/Soil Quality

Fertiliser remains a critical component to ensure high crop yields - especially nitrogen fertiliser.

# Import/Export

International links are essential for our food security. Large parts of the fertiliser supply chain in the UK rely on international exports - natural gas and even raw ammonia are typically shipped in from the US/EU. Therefore modelling the effect of supply disruption, due to shipping delays, trade policy ect is vital to understand the UK's food security.

# Supply/Demand and economics effects

An attractive projected wheat price will encourage farmers to plant, increasing the supply of wheat and reducing the imbalance. We would expect this imbalance to reduce over time as market participants react to the changing conditions.